In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Set the executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
INFO:WDM:Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
INFO:WDM:Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\lizly\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\lizly\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [2]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [3]:
# HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [4]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA Readies Perseverance Mars Rover's Earthly Twin "

In [5]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape."

### Featured Images

In [6]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [7]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [8]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [9]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [10]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

## Mars Facts

In [11]:
# Adding HTML to a DataFrame
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [12]:
# Convert df back to html
df.to_html

<bound method DataFrame.to_html of                                     Mars            Earth
description                                              
Mars - Earth Comparison             Mars            Earth
Diameter:                       6,779 km        12,742 km
Mass:                    6.39 × 10^23 kg  5.97 × 10^24 kg
Moons:                                 2                1
Distance from Sun:        227,943,824 km   149,598,262 km
Length of Year:           687 Earth days      365.24 days
Temperature:                -87 to -5 °C      -88 to 58°C>

## D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [13]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'
browser.visit(url)

In [14]:
# Parser
html = browser.html
browser.visit(url)
html_soup = soup(html, 'html.parser')

In [15]:
# 2. Create a list to hold the images and titles.
hemisphere_image_titles = []
hemisphere_image_urls = []


# 3. Write code to retrieve the image urls and titles for each hemisphere.
mars_hemisphere_html = browser.html
mars_hemisphere_soup = soup(mars_hemisphere_html,"html.parser")
mars_hemispheres =  mars_hemisphere_soup.find_all("div", class_ = "item")

# Loop through the full-resolution image URL
for item in mars_hemispheres:
    
    # Create an empty dictionary
    hemispheres = {}
    
    link = item.find("a",class_="itemLink product-item").get("href")
    hemisphere_url = url + link
    print(hemisphere_url)
    
    browser.visit(hemisphere_url)
    hemisphere_html = browser.html
    hemisphere_soup = soup(hemisphere_html,"html.parser")
    
    # Image 
    image = hemisphere_soup.find("img",class_ = "wide-image").get("src")
    
    # Title
    title = hemisphere_soup.find("h2",class_ = "title").text
    
    # Add image urls to hemispheres dictionary
    hemispheres["title"] = title
    hemispheres["img_url"] = f'{url}{image}'
    hemisphere_image_urls.append(hemispheres)
    

https://marshemispheres.com/cerberus.html
https://marshemispheres.com/schiaparelli.html
https://marshemispheres.com/syrtis.html
https://marshemispheres.com/valles.html


In [16]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [ ]:
# 5. Quit the browser
browser.quit()